In [1]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-06-20 08:04:54--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  1.22MB/s    in 0.9s    

2023-06-20 08:04:55 (1.22 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [8]:
with open('input.txt', 'r') as f:
    text = f.read()

In [12]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [22]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [110]:
# create mapping from char to intrgers
stoi = { ch: i for i, ch in enumerate(chars) }
itos = { i: ch for ch, i in stoi.items() }

encode = lambda s: [ stoi[c] for c in s ]
decode = lambda n: ''.join(itos[i] for i in n)

In [67]:
# encode text data
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [72]:
# split train and validation
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [86]:
block_size = 8  # a.k.a context length
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [114]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for i in range(len(x)):
    context = x[:i+1]
    target = y[i]
    print(f' when context is {decode(context.tolist())} target is {decode([target.tolist()])}')

 when context is F target is i
 when context is Fi target is r
 when context is Fir target is s
 when context is Firs target is t
 when context is First target is  
 when context is First  target is C
 when context is First C target is i
 when context is First Ci target is t


In [383]:
# create batches of data
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    # generate small batches
    data = train_data if split == 'train' else val_data    
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

In [384]:
xb, yb = get_batch('train')

In [442]:
# try bigram
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanquageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets=None):
        
        # (B,T) integers
        logits = self.token_embedding_table(idx)  # (B,T,C)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(-1)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices
        for _ in range(max_new_tokens):
            # predict
            logits, loss = self(idx)  # (B, T, C)
            # get last time step
            logits = logits[:,-1,:]  # (B, C)
            # turn to probabilities
            probs = F.softmax(logits, dim=1)  # (B, C)
            # sample from distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # append ampled to idx
            idx = torch.concat((idx, idx_next), dim=1)  # (B, T+1)
        return idx
            
m = BigramLanquageModel(vocab_size)
out, loss = m(xb, yb)
print(out.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1,1), dtype=torch.long), max_new_tokens=150).tolist()[0]))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJt-wBpm&yiltNCjeO3:Cx&vvMYW-txjuAd IRFbTpJ$zkZelxZt


In [462]:
# create a pytorch optimizer
optimizer = torch.optim.Adam(m.parameters(), lr=1e-3)

In [457]:
batch_size = 32

for step in range(100):
    # get batch
    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    print(loss.item())
    loss.backward()
    optimizer.step()

2.43640398979187
2.426898717880249
2.5075273513793945
2.463148593902588
2.5462875366210938
2.463998556137085
2.5090854167938232
2.452609062194824
2.5705862045288086
2.455322265625
2.512864828109741
2.4144763946533203
2.5003230571746826
2.4841244220733643
2.4182634353637695
2.226856231689453
2.4743053913116455
2.472527503967285
2.4403815269470215
2.5256049633026123
2.497192621231079
2.3949100971221924
2.5532591342926025
2.4884181022644043
2.493783950805664
2.431213140487671
2.46244478225708
2.535665512084961
2.312112808227539
2.415705680847168
2.3488693237304688
2.2858898639678955
2.51921010017395
2.5707833766937256
2.3125061988830566
2.5066559314727783
2.382559061050415
2.3821587562561035
2.4549572467803955
2.4751644134521484
2.489253282546997
2.4664511680603027
2.607337236404419
2.555123805999756
2.450185537338257
2.3522191047668457
2.403778076171875
2.435080051422119
2.4556479454040527
2.4787862300872803
2.4972681999206543
2.4047818183898926
2.486809253692627
2.6673214435577393
2.483

2.560412645339966
2.51162052154541
2.439964771270752
2.4837682247161865
2.4865634441375732
2.6679298877716064
2.358966112136841
2.426464319229126
2.5408010482788086
2.5243630409240723
2.5636749267578125
2.4862964153289795
2.445453643798828
2.501800060272217
2.4443392753601074
2.472257375717163
2.5400259494781494
2.388411045074463
2.3927996158599854
2.3848488330841064
2.419313907623291
2.4796030521392822
2.437469720840454
2.4426097869873047
2.4553723335266113
2.497819423675537
2.430159091949463
2.3793578147888184
2.4488112926483154
2.506166458129883
2.5002920627593994
2.5194056034088135
2.4849634170532227
2.4301633834838867
2.432321786880493
2.3969180583953857
2.441269636154175
2.3602523803710938
2.647460699081421
2.4452993869781494
2.392061948776245
2.5575907230377197
2.491698741912842
2.4445459842681885
2.474050283432007
2.453695297241211
2.4793615341186523
2.4412524700164795
2.409463405609131
2.5006308555603027
2.4491207599639893
2.4369423389434814
2.490516424179077
2.298324346542358

2.2757368087768555
2.3684303760528564
2.5446617603302
2.4962716102600098
2.3408727645874023
2.417057514190674
2.496151924133301
2.4551210403442383
2.3408775329589844
2.3574018478393555
2.449544906616211
2.4341630935668945
2.4511280059814453
2.453908681869507
2.4099268913269043
2.4733684062957764
2.4459755420684814
2.5496528148651123
2.467470169067383
2.483883857727051
2.464942455291748
2.376004457473755
2.557920217514038
2.404022455215454
2.4336936473846436
2.2486016750335693
2.3884873390197754
2.4114866256713867
2.459228992462158
2.5107028484344482
2.5030782222747803
2.440615653991699
2.3470518589019775
2.576277732849121
2.489084243774414
2.4223546981811523
2.370388984680176
2.3914504051208496
2.4954307079315186
2.504133701324463
2.435999631881714
2.4180524349212646
2.3919122219085693
2.4250106811523438
2.4273009300231934
2.485071897506714
2.5235540866851807
2.444688320159912
2.4937853813171387
2.4657440185546875
2.4361531734466553
2.474217653274536
2.382319211959839
2.504651546478271

2.420198440551758
2.4704461097717285
2.5177085399627686
2.341848134994507
2.463834285736084
2.4254310131073
2.497419595718384
2.4207682609558105
2.336519956588745
2.368435859680176
2.5643749237060547
2.4099698066711426
2.4185009002685547
2.440584421157837
2.4024839401245117
2.619866132736206
2.339045763015747
2.4285202026367188
2.4675958156585693
2.448634386062622
2.5424578189849854
2.465423107147217
2.3562355041503906
2.4669222831726074
2.4156277179718018
2.329353094100952
2.3559882640838623
2.5016024112701416
2.542987108230591
2.491936206817627
2.492199659347534
2.503565788269043
2.4335920810699463
2.525804281234741
2.369086503982544
2.3966827392578125
2.3926844596862793
2.4526004791259766
2.5303170680999756
2.3449833393096924
2.429351806640625
2.427381753921509
2.4301791191101074
2.4923300743103027
2.5729427337646484
2.485570192337036
2.5897254943847656
2.319014310836792
2.5588760375976562
2.628446340560913
2.493898391723633
2.3756468296051025
2.527918577194214
2.4378066062927246
2.

2.3964717388153076
2.3750224113464355
2.667664051055908
2.2896738052368164
2.551525831222534
2.506972074508667
2.4827334880828857
2.394319534301758
2.4953958988189697
2.40968656539917
2.4594435691833496
2.4259727001190186
2.6008968353271484
2.419553518295288
2.305795431137085
2.625955104827881
2.3365609645843506
2.4761695861816406
2.4162354469299316
2.3967318534851074
2.458024024963379
2.3471872806549072
2.4412407875061035
2.6499011516571045
2.364349842071533
2.4679365158081055
2.4931273460388184
2.3888325691223145
2.3710904121398926
2.4656758308410645
2.4272146224975586
2.376494884490967
2.481844186782837
2.424280881881714
2.5312392711639404
2.47758150100708
2.3845953941345215
2.4529225826263428
2.588895320892334
2.3653247356414795
2.452118396759033
2.571106195449829
2.5054264068603516
2.5571141242980957
2.391096353530884
2.443896532058716
2.457183361053467
2.4511654376983643
2.488182306289673
2.5044233798980713
2.3593180179595947
2.440805196762085
2.2652523517608643
2.390446901321411

...

2.6250946521759033
2.482107162475586
2.379754066467285
2.3503663539886475
2.417368173599243
2.4663329124450684
2.4773976802825928
2.24219012260437
2.498103141784668
2.4987645149230957
2.5727789402008057
2.4486260414123535
2.4708173274993896
2.3919718265533447
2.454299211502075
2.492867946624756
2.4703500270843506
2.5425865650177
2.532921075820923
2.37093186378479
2.364809036254883
2.427581548690796
2.4424779415130615
2.505213737487793
2.4140853881835938
2.526400089263916
2.486459255218506
2.3251588344573975
2.4360196590423584
2.4388267993927
2.4861345291137695
2.446420431137085
2.4493556022644043
2.415381669998169
2.6224968433380127
2.4935412406921387
2.4679296016693115
2.464033365249634
2.389617681503296
2.516357660293579
2.4338550567626953
2.352261543273926
2.4548897743225098
2.589587688446045
2.4614861011505127
2.31801176071167
2.6054437160491943
2.467569351196289
2.2989354133605957
2.4709603786468506
2.4162089824676514
2.4619264602661133
2.5432403087615967
2.4402053356170654
2.4439

2.4752626419067383
2.4716057777404785
2.4492650032043457
2.2483222484588623
2.515822649002075
2.404470205307007
2.506891965866089
2.441239356994629
2.4204518795013428
2.433194875717163
2.520728349685669
2.3544392585754395
2.37721586227417
2.6431565284729004
2.4201436042785645
2.3911216259002686
2.3995983600616455
2.511880874633789
2.4483203887939453
2.5010361671447754
2.463719367980957
2.524157762527466
2.4103333950042725
2.5173299312591553
2.4171366691589355
2.395490884780884
2.5060362815856934
2.4024038314819336
2.469633102416992
2.618661642074585
2.4640626907348633
2.4343607425689697
2.3115575313568115
2.571296453475952
2.3871095180511475
2.4049489498138428
2.4255142211914062
2.4487216472625732
2.5733275413513184
2.4803197383880615
2.4589734077453613
2.3886044025421143
2.5730090141296387
2.5050930976867676
2.4955172538757324
2.5124642848968506
2.480139970779419
2.4516477584838867
2.4368066787719727
2.433397054672241
2.555426597595215
2.4006049633026123
2.461275577545166
2.5316836833

2.5096490383148193
2.3794398307800293
2.609088659286499
2.5277554988861084
2.303452253341675
2.518700122833252
2.3387036323547363
2.5128848552703857
2.4381940364837646
2.6148972511291504
2.483837127685547
2.527982234954834
2.4538536071777344
2.41500186920166
2.4696009159088135
2.3939270973205566
2.4864726066589355
2.521381378173828
2.52087664604187
2.391433000564575
2.4585297107696533
2.362166166305542
2.38263201713562
2.417189836502075
2.5198936462402344
2.386981725692749
2.4238555431365967
2.363851547241211
2.3838553428649902
2.635608673095703
2.365957021713257
2.4036128520965576
2.4870457649230957
2.518587589263916
2.3999416828155518
2.510998487472534
2.5064690113067627
2.460904836654663
2.2399067878723145
2.4031970500946045
2.4907357692718506
2.5120537281036377
2.373849868774414
2.420618772506714
2.4355313777923584
2.452702522277832
2.3845014572143555
2.321115016937256
2.5212485790252686
2.3312623500823975
2.5718562602996826
2.3841664791107178
2.4616878032684326
2.3761684894561768


2.349482774734497
2.5244572162628174
2.4848713874816895
2.3695991039276123
2.458665609359741
2.572676181793213
2.392404794692993
2.404545307159424
2.5492162704467773
2.4141669273376465
2.448537826538086
2.373194456100464
2.469924211502075
2.4727985858917236
2.45737886428833
2.3782970905303955
2.4605681896209717
2.4797580242156982
2.3986568450927734
2.369086980819702
2.533688545227051
2.492779016494751
2.460289478302002
2.3003644943237305
2.252075433731079
2.5174107551574707
2.5442659854888916
2.4572155475616455
2.3143720626831055
2.506528377532959
2.5895698070526123
2.520066261291504
2.492525339126587
2.450183391571045
2.5357837677001953
2.3620011806488037
2.4219536781311035
2.3867886066436768
2.4929723739624023
2.448314666748047
2.4033637046813965
2.3262765407562256
2.4490573406219482
2.5537521839141846
2.441187620162964
2.48492693901062
2.4657177925109863
2.4934332370758057
2.323179244995117
2.3367700576782227
2.4989356994628906
2.4767277240753174
2.4786674976348877
2.557973623275757

2.410095453262329
2.446112632751465
2.4638898372650146
2.384268283843994
2.5492701530456543
2.5283937454223633
2.4156858921051025
2.489215850830078
2.4045474529266357
2.4559521675109863
2.5970029830932617
2.4319770336151123
2.4663233757019043
2.3995275497436523
2.4151525497436523
2.5671794414520264
2.517277240753174
2.4675304889678955
2.4596598148345947
2.44671630859375
2.5820624828338623
2.4074923992156982
2.378981113433838
2.4419732093811035
2.5427682399749756
2.4140913486480713
2.3743157386779785
2.4674642086029053
2.4790492057800293
2.3890011310577393
2.359421491622925
2.4413902759552
2.585137367248535
2.441789150238037
2.4087038040161133
2.4370267391204834
2.3661422729492188
2.5188748836517334
2.4681460857391357
2.3408241271972656
2.4852986335754395
2.4763667583465576
2.420236587524414
2.4835965633392334
2.5257937908172607
2.465542793273926
2.420715808868408
2.462001085281372
2.3659653663635254
2.5575945377349854
2.3711097240448
2.436262369155884
2.4047019481658936
2.4815530776977

In [461]:
print(decode(m.generate(idx = torch.zeros((1,1), dtype=torch.long), max_new_tokens=150).tolist()[0]))


Yoke s,
Wh pe dy,
Telealdoronsbl ce l g qusalerf me.
Heeacer waprme qu qun sed m; giteserm faleshinelly the f h
A f uneris thyewerer tuerarut, p try o
